In [2]:
import pandas as pd
import sklearn.linear_model as ln
import matplotlib.pyplot as plt

df = pd.read_csv("Статистика за 2018 год.csv", encoding= 'windows-1251')
df.head()

,Число бракованных листов,Скорость прокатки,Марка стали
0,3,3.8,A
1,3,3.8,B
2,3,4.8,B
3,4,4.4,B
4,6,5.0,B


In [3]:
df.shape

(237, 3)

In [4]:
df = df.rename(columns = {"Число бракованных листов": "deffect", "Скорость прокатки": "speed", "Марка стали": "mark"})
df.head()

,deffect,speed,mark
0,3,3.8,A
1,3,3.8,B
2,3,4.8,B
3,4,4.4,B
4,6,5.0,B


Беглый анализ пристальным взглядом показывает, что даннные в порядке и дальнейшая преподготовка не нужна (отсутствие NA, диапазон скоростей в норме, марки всего две)

In [5]:
df_a = df.query("mark == 'A'")
df_a.describe()

,deffect,speed
count,139.000000,139.000000
mean,3.179856,4.475540
std,1.889144,0.684703
min,0.000000,2.800000
25%,2.000000,4.000000
50%,3.000000,4.600000
75%,4.000000,4.900000
max,8.000000,6.200000


In [20]:
df_a = df.query("mark == 'A' and deffect > 3")
tmp_a = df_a.describe()
tmp_a

,deffect,speed
count,53.000000,53.000000
mean,5.113208,4.530189
std,1.339638,0.763238
min,4.000000,2.800000
25%,4.000000,4.000000
50%,5.000000,4.600000
75%,6.000000,5.000000
max,8.000000,6.200000


In [25]:
count_a = tmp_a.deffect.iloc[0]
mean_a = tmp_a.deffect.iloc[1]
std_a = tmp_a.deffect.iloc[2]
print(count_a)
print(mean_a)
print(std_a)

53.0
5.113207547169812
1.3396379728880052


In [63]:
df_b = df.query("mark == 'B'")
df_b.describe()

,deffect,speed
count,98.000000,98.000000
mean,3.367347,4.558163
std,1.689188,0.713048
min,0.000000,2.500000
25%,2.000000,4.100000
50%,3.000000,4.600000
75%,4.000000,5.000000
max,8.000000,6.500000


In [22]:
df_b = df.query("mark == 'B' and deffect > 3")
tmp_b = df_b.describe()
tmp_b

,deffect,speed
count,44.000000,44.000000
mean,4.886364,4.443182
std,1.104239,0.624462
min,4.000000,3.000000
25%,4.000000,4.100000
50%,4.000000,4.450000
75%,6.000000,4.825000
max,8.000000,6.000000


In [26]:
count_b = tmp_b.deffect.iloc[0]
mean_b = tmp_b.deffect.iloc[1]
std_b = tmp_b.deffect.iloc[2]
print(count_b)
print(mean_b)
print(std_b)

44.0
4.886363636363637
1.1042393802430215


In [27]:
from scipy.stats import ttest_ind_from_stats
ttest_ind_from_stats(mean_a, std_a, count_a, mean_b, std_b, count_b)

Ttest_indResult(statistic=0.8979646072876029, pvalue=0.3714746018609977)

P.value > 0.05 Принимаем нулевую гипотезу о равенстве средних в выборках

Очевидно, что тут какая-то ошибка насчёт статистического различия в марках A и B, ибо пытаться доказать, что 53/139 ~ 0.38 больше, чем 44/98 ~ 0.45 нет никакого смысла

Тем не менее, для статистического различия выборок в скоростях всё ОК.

In [29]:
df_high_speed = df.query("deffect > 3 and speed > 4")
df_high_speed.describe()

,deffect,speed
count,74.000000,74.000000
mean,5.054054,4.783784
std,1.203783,0.493250
min,4.000000,4.100000
25%,4.000000,4.425000
50%,5.000000,4.700000
75%,6.000000,5.000000
max,8.000000,6.200000


In [28]:
df_high_speed = df.query("deffect > 3 and speed <= 4")
df_high_speed.describe()

,deffect,speed
count,23.000000,23.000000
mean,4.869565,3.547826
std,1.358621,0.348855
min,4.000000,2.800000
25%,4.000000,3.300000
50%,4.000000,3.600000
75%,5.000000,3.800000
max,8.000000,4.000000


In [92]:
from scipy.stats import chisquare
chisquare([74, 23], [48.5, 48.5])

Power_divergenceResult(statistic=47.847682119205295, pvalue=4.6065135823396e-12)

p-value < 0.05 значит различие в долях статистически значимо. К слову, всего у нас 151 наблюдение для партий с браком > 3.
75,5 = 151 / 2 — ожидаемая частота наблюдений для каждой из подгрупп, если бы скорость проката ни на что не влияла
https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.stats.chisquare.html#scipy.stats.chisquare